In [58]:
import operator as op
import numpy as np
import pandas as pd
import requests
import param
import panel as pn
import hvplot.pandas
import hvplot.streamz
import holoviews as hv
from holoviews.element.tiles import EsriImagery
from holoviews.selection import link_selections
from datashader.utils import lnglat_to_meters
from streamz.dataframe import PeriodicDataFrame
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import re

def weather_data(cities, openweathermap_api_key='0d19335cb8ac57d7f05bda57a5de0433'):
    """
    Get weather data for a list of cities using the openweathermap API
    """
    L = []
    for c in cities:
        res = requests.get(f'http://api.openweathermap.org/data/2.5/weather?q={c}&appid={openweathermap_api_key}&units=imperial')
        L.append(res.json())

    df = pd.DataFrame(L)
    df['lon'] = df['coord'].map(op.itemgetter('lon'))
    df['lat'] = df['coord'].map(op.itemgetter('lat'))
    df['Temprature'] = int(round(df['main'].map(op.itemgetter('temp')),0))
    df['Humidity'] = df['main'].map(op.itemgetter('humidity'))
    df['Wind_Speed'] = df['wind'].map(op.itemgetter('speed'))
    return df[['name','lon', 'lat','Temprature','Humidity','Wind_Speed']]

    
def streaming_weather_data(**kwargs):
    """
    callback function 
    get San Francisco weather data 
    """
    df = weather_data(['San Francisco'])
    df['time'] = [pd.Timestamp.now()]
    return df.set_index('time')

# Make a streaming dataframe 
df = PeriodicDataFrame(streaming_weather_data, interval='1s')

#print(df['Wind_Speed'])
#print(df)

#print(str(df['Wind_Speed']).split("\t")[0])

#print(''.join(filter(str.isnumeric, str(df.Wind_Speed))))

fig = go.Figure(go.Indicator(
    mode = "number",
    value = int(''.join(filter(str.isnumeric, str(df.Temprature.sum())))),
    title = {'text': "Reps"}
))

fig.update_layout(
    paper_bgcolor="white",
    height=200,  # Added parameter
    width=200
)

np_label=pn.Column(
    pn.Row('#Your Current Reps'),
    pn.Row(df[['Wind_Speed']].hvplot.line(title='Humidity',backlog=1000),fig,sizing_mode='stretch_width')
)

np_label.servable()

# pn_realtime = pn.Column(
#     pn.Row(
#       df[['Humidity']].hvplot.line(title='Humidity', backlog=1000)),
#       #df[['Wind Speed']].hvplot.line(title='Wind Speed', backlog=1000),
#       #df[['Wind Speed']].hvplot.table(title='TEST', backlog=1000),
#       #print(int(str(df.Temprature.sum())[-6:-3]))
#       fig.show()
# )


# pn_realtime = pn.Column(
#     pn.Row(
#       df[['Temprature']].hvplot.line(title='Temprature', backlog=1000),
#       df[['Humidity']].hvplot.line(title='Humidity', backlog=1000)),
#   df[['Wind Speed']].hvplot.line(title='Wind Speed', backlog=1000)
# )

#pn_realtime.servable()

Column
    [0] Row
        [0] Markdown(str)
    [1] Row(sizing_mode='stretch_width')
        [0] HoloViews(DynamicMap)
        [1] Plotly(Figure)